### Import Packages

In [1]:
import os
import os.path
import codecs
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from gensim import matutils, models
from gensim.models.coherencemodel import CoherenceModel
import scipy.sparse
import nltk
from gensim import corpora
from gensim.models import LsiModel
from lda import guidedlda as glda
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andreasabia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Import Data

In [2]:
os.chdir("/Users/...")
dir = '/Users/...'
fileList = os.listdir(dir)
newfileList=[]
for i in fileList:
    if '.txt' in i:
        newfileList.append(i)
fileList=newfileList

In [3]:
docs_list = []
for i in fileList:
    with open(i) as f:
        int_step = f.read()
    docs_list.append(int_step)

In [4]:
df = pd.DataFrame(list(zip(fileList, docs_list)), 
               columns =['title', 'body'])

In [5]:
article_title=[]
for i in fileList:
    with open(i) as f:
        int_step= f.readline()
    article_title.append(int_step)
df['Article_Title']=article_title    

In [6]:
df['File_Name']=df.title
new_title=[]
for i in df['File_Name']:
    p= i.split(".")[0]
    p= p.replace("-"," ")
    p= p.replace("_"," ")
    p= p.replace("copy","")
    p= p.replace ("(1)","")
    new_title.append(p)
    
df['File_Name']=new_title

In [7]:
df.drop(columns=["title"], inplace=True)

In [8]:
df

,body,Article_Title,File_Name
0,Manufacturing Intelligence: Keep Your Processe...,Manufacturing Intelligence: Keep Your Processe...,wp manufacturing process optimization final
1,Data Management for Advanced Analytics\nBEST P...,Data Management for Advanced Analytics\n,TIBCO Custom Version TDWI BPReport Q220
2,Special Challenges in B2B Counterparty Master ...,Special Challenges in B2B Counterparty Master ...,WP special challenges b2b counterparty
3,The four Rs of building your reference data ma...,The four Rs of building your reference data ma...,final final WP four Rs of building your refere...
4,Code-Free and Code-Based Data Science with Rap...,Code-Free and Code-Based Data Science with Rap...,Code free Code based DS
...,...,...,...
161,Cloud-based AI and ML: Top Considerations and ...,Cloud-based AI and ML: Top Considerations and ...,WP cloud based AI and ML
162,Azure Databricks: The Best Platform to Run ML ...,Azure Databricks: The Best Platform to Run ML ...,Azure Databricks The Best Platform to Run ML a...
163,How to Take AI Projects From Start to Win\n\n4...,How to Take AI Projects From Start to Win\n,ai projects from start to win 110730
164,The 3 R’s of AI Adoption: Refactor. Reinvent. ...,The 3 R’s of AI Adoption: Refactor. Reinvent. ...,ai adoption 109891


### Clean Data

In [9]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = re.sub(r'\b[A-Z]+\b', '', text)
    text = re.sub(r'\b[a-zA-Z]{1}\b','', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\u2002', '', text)
    text = re.sub('\x0c', '', text)
    text = text.lower()
    text = text.strip()
    text = text.lstrip()
    return text

def lemmatize(text):
    lem_sentence=[]
    tokenize_text=word_tokenize(text)
    tagged_text=pos_tag(tokenize_text)
    for i in tagged_text:
        if i[1]=='VBG':
            posizione='v'
        elif i[1] in ('NN', 'NNS'):
            posizione='n'
        elif i[1] in ('JJ', 'JJR'):
            posizione='a'
        else:
            continue
        lem_sentence.append(wordnet_lemmatizer.lemmatize(i[0], pos=posizione))
    return ' '.join(lem_sentence)

def clean_title(text):
    text = re.sub('\n', '', text)
    text = text.lower()
    text = text.strip()
    text = text.lstrip()
    return text


def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)
    
def introduction(text):
    tokens=word_tokenize(text)
    lungo=len(tokens)
    counter=0
    new_sentence=[]
    for i in range(len(tokens)):
        if i <= min(200, lungo):
            new_sentence.append(tokens[counter])
            counter=counter+1
        else:
            continue
    return ' '.join(new_sentence)
        
        
round1 = lambda x: clean_text_round1(x)
round2 = lambda x:clean_title(x)

In [10]:
df['Article_Title']=pd.DataFrame(df["Article_Title"].apply(round2))
df.drop_duplicates(subset=['Article_Title'], inplace=True)
df['Whitepaper'] = pd.DataFrame(df.body.apply(round1))
df['Whitepaper'] = pd.DataFrame(df.Whitepaper.apply(lemmatize))
df['Introduction']=pd.DataFrame(df.Whitepaper.apply(introduction))

In [11]:
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said',
                 'dataiku','contactdataikucom','wwwdataikucom',
                 'alteryx', 'datarobot', 'spark', 'rapidminer', 'databricks', 'azure', 'apache'
                 'domino','hadoop','rapidmin', 'wwwtibcocom','data']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

In [12]:
tokenized_texts=[]
for i in df.Whitepaper:
    tokens=word_tokenize(i)
    int_step=[]
    for token in tokens:
        if token not in stop_words:
            int_step.append(token)
        else:
            continue
    tokenized_texts.append(int_step)
df['Tokenized_Whitepaper']=tokenized_texts

In [13]:
data=df.copy()

In [14]:
data

,body,Article_Title,File_Name,Whitepaper,Introduction,Tokenized_Whitepaper
0,Manufacturing Intelligence: Keep Your Processe...,manufacturing intelligence: keep your processe...,wp manufacturing process optimization final,manufacture intelligence process controldriver...,manufacture intelligence process controldriver...,"[manufacture, intelligence, process, controldr..."
1,Data Management for Advanced Analytics\nBEST P...,data management for advanced analytics,TIBCO Custom Version TDWI BPReport Q220,data management advanced analytics philip russ...,data management advanced analytics philip russ...,"[management, advanced, analytics, philip, russ..."
2,Special Challenges in B2B Counterparty Master ...,special challenges in b2b counterparty master ...,WP special challenges b2b counterparty,special challenge counterparty master data man...,special challenge counterparty master data man...,"[special, challenge, counterparty, master, man..."
3,The four Rs of building your reference data ma...,the four rs of building your reference data ma...,final final WP four Rs of building your refere...,r build reference data management business cas...,r build reference data management business cas...,"[r, build, reference, management, business, ca..."
4,Code-Free and Code-Based Data Science with Rap...,code-free and code-based data science with rap...,Code free Code based DS,codefree data science rapidminer data science ...,codefree data science rapidminer data science ...,"[codefree, science, science, team, sport, skil..."
...,...,...,...,...,...,...
161,Cloud-based AI and ML: Top Considerations and ...,cloud-based ai and ml: top considerations and ...,WP cloud based AI and ML,top consideration benefit youre think right ti...,top consideration benefit youre think right ti...,"[consideration, benefit, artificial, intellige..."
162,Azure Databricks: The Best Platform to Run ML ...,azure databricks: the best platform to run ml ...,Azure Databricks The Best Platform to Run ML a...,azure platform organization look analytics bus...,azure platform organization look analytics bus...,"[platform, organization, look, analytics, busi..."
163,How to Take AI Projects From Start to Win\n\n4...,how to take ai projects from start to win,ai projects from start to win 110730,project start pillar start program executive s...,project start pillar start program executive s...,"[project, start, pillar, start, program, execu..."
164,The 3 R’s of AI Adoption: Refactor. Reinvent. ...,the 3 r’s of ai adoption: refactor. reinvent. ...,ai adoption 109891,adoption refactor reinvent reimagine kimberly ...,adoption refactor reinvent reimagine kimberly ...,"[adoption, refactor, reinvent, reimagine, kimb..."


In [17]:
os.chdir("/Users/...")
data.to_csv('Thesis_data.csv')